In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range = 0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='QagoRBAos7umhwlwMH7KObYq7OpyxOQ9L1MV25zVnydD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'cnn-donotdelete-pr-zwwnova430gtni'
object_key = 'Foods.zip'

streaming_body_8 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [11]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_8.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [12]:
pwd

'/home/wsuser/work'

In [14]:
import os
filenames = os.listdir('/home/wsuser/work/Foods/Training')

In [27]:
x_train =  train_datagen.flow_from_directory('/home/wsuser/work/Foods/Training',
                                           target_size=(64,64),
                                           class_mode='categorical',
                                           batch_size=100)
x_test =  test_datagen.flow_from_directory('/home/wsuser/work/Foods/Testing',
                                           target_size=(64,64),
                                           class_mode='categorical',
                                           batch_size=100)

Found 2710 images belonging to 8 classes.
Found 686 images belonging to 8 classes.


In [28]:
x_train.class_indices

{'Apple': 0,
 'Banana': 1,
 'Bhel puri': 2,
 'Burger': 3,
 'Egg': 4,
 'Mango': 5,
 'Onion': 6,
 'Pizza': 7}

In [29]:
model = Sequential()

In [30]:
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3))) 

In [31]:
model.add(MaxPooling2D(pool_size=(2,2))) 

In [32]:
model.add(Flatten())

In [33]:
model.add(Dense(300,activation='relu')) 
model.add(Dense(150,activation='relu')) 
model.add(Dense(8,activation='softmax')) 

In [34]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
model.fit_generator(x_train,
                    steps_per_epoch=len(x_train),
                    epochs=10,
                    validation_data=x_test,
                    validation_steps=len(x_test))

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
28/28 [==============================] - 89s 3s/step - loss: 2.2397 - accuracy: 0.2314 - val_loss: 1.5222 - val_accuracy: 0.3703
Epoch 2/10
28/28 [==============================] - 84s 3s/step - loss: 1.4649 - accuracy: 0.4517 - val_loss: 1.1958 - val_accuracy: 0.5423
Epoch 3/10
28/28 [==============================] - 84s 3s/step - loss: 1.2607 - accuracy: 0.5413 - val_loss: 1.0505 - val_accuracy: 0.6137
Epoch 4/10
28/28 [==============================] - 86s 3s/step - loss: 1.0909 - accuracy: 0.5926 - val_loss: 1.1080 - val_accuracy: 0.6064
Epoch 5/10
28/28 [==============================] - 84s 3s/step - loss: 1.0769 - accuracy: 0.5952 - val_loss: 1.0404 - val_accuracy: 0.6283
Epoch 6/10
28/28 [==============================] - 84s 3s/step - loss: 0.9722 - accuracy: 0.6480 - val_loss: 0.9171 - val_accuracy: 0.6808
Epoch 7/10
28/28 [==============================] - 84s 3s/step - loss: 0.8976 - accuracy: 0.6786 - val_loss: 0.8889 - val_accuracy: 0.6924
Epoch 8/10
28/28 [==

In [36]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=5)

lr = ReduceLROnPlateau(monitor='val_accuracy', 
                       factor=0.5, 
                       patience=5,
                       min_lr=0.00001)

callbacks = [early_stop,lr]
# Training model

model.fit_generator(x_train,
                    steps_per_epoch=len(x_train),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=x_test,
                    validation_steps=len(x_test),)

Epoch 1/100
28/28 [==============================] - 86s 3s/step - loss: 0.7451 - accuracy: 0.7339 - val_loss: 0.7780 - val_accuracy: 0.7332
Epoch 2/100
28/28 [==============================] - 85s 3s/step - loss: 0.6948 - accuracy: 0.7576 - val_loss: 0.8551 - val_accuracy: 0.7026
Epoch 3/100
28/28 [==============================] - 85s 3s/step - loss: 0.6608 - accuracy: 0.7594 - val_loss: 0.9035 - val_accuracy: 0.7143
Epoch 4/100
28/28 [==============================] - 85s 3s/step - loss: 0.6680 - accuracy: 0.7587 - val_loss: 0.8050 - val_accuracy: 0.7347
Epoch 5/100
28/28 [==============================] - 85s 3s/step - loss: 0.6148 - accuracy: 0.7712 - val_loss: 0.8665 - val_accuracy: 0.7114
Epoch 6/100
28/28 [==============================] - 84s 3s/step - loss: 0.6037 - accuracy: 0.7775 - val_loss: 0.8538 - val_accuracy: 0.7216
Epoch 7/100
28/28 [==============================] - 84s 3s/step - loss: 0.5776 - accuracy: 0.7871 - val_loss: 0.8991 - val_accuracy: 0.7143
Epoch 8/100
2

In [37]:
model.save("nutrition.h5")

In [38]:
!tar -zcvf image-classification-model_new.tgz nutrition.h5

nutrition.h5


In [39]:
ls -1

Foods/
image-classification-model_new.tgz
nutrition.h5


In [40]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 20.0 MB/s eta 0:00:01


In [42]:
import numpy as np
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "Jy4yk-xqCXg6sBehmBlOk4VpwoA1Cy9QQnkldHJzFO59"
}
client = APIClient(wml_credentials)

In [ ]:
client = APIClient(wml_credentials)

In [43]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [44]:
space_uid = guid_from_space_name(client, 'Image Classification')
print("Space UID = " + space_uid)

Space UID = 5f96af49-c412-4caf-90a1-4370c10f3e0c


In [45]:
client.set.default_space(space_uid)

'SUCCESS'

In [47]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [56]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [58]:
model_details = client.repository.store_model(model = 'image-classification-model_new.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [59]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-15T04:48:47.144Z',
  'id': '963f2f54-c2c9-434b-989d-75444e9970f7',
  'modified_at': '2022-11-15T04:48:54.722Z',
  'name': 'CNN',
  'owner': 'IBMid-667000CBKP',
  'resource_key': '4301a7ab-cf7a-4c17-b6bb-d725fe5ef317',
  'space_id': '5f96af49-c412-4caf-90a1-4370c10f3e0c'},
 'system': {'warnings': []}}

In [60]:
model_id = client.repository.get_model_id(model_details)
model_id

'963f2f54-c2c9-434b-989d-75444e9970f7'

In [72]:
client.repository.download(model_id,'IBM_model_deploy.tgz')

Successfully saved model content to file: 'IBM_model_deploy.tgz'


'/home/wsuser/work/IBM_model_deploy.tgz'